In [24]:
# import basic librairies
import os
import pandas as pd
import datetime
import numpy as np
import calendar
import holidays

# import utils
import holidays
from timeit import default_timer as timer

# import preprocessing functions
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

# import pipeline functions
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# import models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LassoLarsCV, ElasticNetCV

# other functions
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

## Load data

In [9]:
df = pd.read_csv(os.path.join('data', 'initial_data.csv'), parse_dates=['DateOfDeparture'])
print('Number of observations: %.f' % (df.shape[0]))

Number of observations: 11128


In [16]:
df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('log_PAX', axis=1), 
                                                    df['log_PAX'], train_size=0.8, 
                                                   random_state=42)

## Create preprocessing functions

In [31]:
def _merge_external_data(X):
    filepath = os.path.join('data', 'external_data.csv')
    
    external_data = pd.read_csv(filepath)
    
    X = X.copy()  # modify a copy of X

    # create a "year" and "month" columns to enable the merge
    X.loc[:, 'DateOfDeparture'] = pd.to_datetime(X.loc[:, 'DateOfDeparture'])
    external_data.loc[:, 'DateOfDeparture'] = pd.to_datetime(external_data.loc[:, 'DateOfDeparture'])
    
    X['connection'] = X['Departure'] + '_' + X['Arrival']
    X['connection'] = ['_'.join(np.sort(x.split('_'))) for x in X['connection']]

    external_data['event_level_dep_arr'] = external_data['event_level_dep'] * external_data['event_level_arr']
    external_data.drop(columns=['event_level_dep', 'event_level_arr'], inplace=True)

    X_merged = X.merge(external_data, how='left', on=['Departure', 'Arrival', 'DateOfDeparture'])
    
    X_merged.drop(columns=['mean_temp_dep', 'departures_performed'], inplace=True)

    return X_merged

In [32]:
def _encode_dates(X):
    X_encoded = X.copy()

    # Make sure that DateOfDeparture is of datetime format
    X_encoded.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_encoded.loc[:, 'DateOfDeparture'])

    # Encode the DateOfDeparture
    #X_encoded.loc[:, 'day'] = X_encoded['DateOfDeparture'].dt.day
    X_encoded.loc[:, 'weekday'] = X_encoded['DateOfDeparture'].dt.weekday
    #X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week
    X_encoded.loc[:, 'is_weekend'] = [True if x in [5, 6] else False for x in X_encoded.loc[:, 'weekday']]
    X_encoded.loc[:, 'n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

    # Cyclical encoding of Dates
    X_encoded.loc[:, 'day_of_year'] = X_encoded['DateOfDeparture'].apply(
        lambda x: (x.date() - datetime.date(x.year, 1, 1)).days)
    
    days_in_year = np.where(X_encoded["year"].apply(lambda y: calendar.isleap(y)), 366, 365)
    X_encoded['sin_doy'] = np.sin(2*np.pi*X_encoded["day_of_year"]/days_in_year)
    X_encoded['cos_doy'] = np.cos(2*np.pi*X_encoded["day_of_year"]/days_in_year)
    X_encoded['sin_dow'] = np.sin(2*np.pi*X_encoded["weekday"]/7)
    X_encoded['cos_dow'] = np.cos(2*np.pi*X_encoded["weekday"]/7)
    
    # Encode holidays
    us_holidays = holidays.US()
    X_encoded.loc[:, 'is_holiday'] = [x in us_holidays for x in X_encoded['DateOfDeparture']]
    X_encoded.loc[:, 'is_beginning_holidays'] = [
        (x not in us_holidays) & (x + datetime.timedelta(days=1) in us_holidays) for x in X_encoded['DateOfDeparture']]
    X_encoded.loc[:, 'is_end_holidays'] = [(x in us_holidays) & (x + datetime.timedelta(days=1) not in us_holidays) for
                                           x in X_encoded['DateOfDeparture']]

    # add distance to closest holidays
    X_encoded = X_encoded.sort_values('DateOfDeparture')
    X_encoded['dumb1'] = X_encoded['dumb2'] = X_encoded["DateOfDeparture"][X_encoded["is_holiday"]]
    X_encoded["dumb1"] = X_encoded["dumb1"].fillna(method="ffill").fillna(method="bfill")
    X_encoded["dumb2"] = X_encoded["dumb2"].fillna(method="bfill").fillna(method="ffill")
    X_encoded["distance_to_previous"] = pd.to_numeric(np.abs(X_encoded["dumb1"] - X_encoded["DateOfDeparture"]).dt.days)
    X_encoded["distance_to_next"] = pd.to_numeric(np.abs(X_encoded["dumb2"] - X_encoded["DateOfDeparture"]).dt.days)
    X_encoded["holidays_distance"] = np.minimum(X_encoded.distance_to_previous, X_encoded.distance_to_next)
    X_encoded.drop(columns=['dumb1', 'dumb2', 'distance_to_previous', 'distance_to_next'], inplace=True)

    X_encoded.drop(columns=['DateOfDeparture', 'day_of_year'], inplace=True)
    
    return X_encoded.sort_index()

# Build the pipeline

In [33]:
def get_estimator(model_type='lgb'):

    # preprocessing the data
    data_merger = FunctionTransformer(_merge_external_data)
    date_encoder = FunctionTransformer(_encode_dates)

    # create a preprocessor
    preprocessor = make_pipeline(data_merger, date_encoder)
    
    # encode categorical columns
    target_cols = ['connection', 'event_level_dep_arr', 'year', 'month']
    categorical_cols = ['Departure', 'Arrival']
    
    encoder = make_column_transformer(
        (OrdinalEncoder(), categorical_cols),
        (TargetEncoder(), target_cols),
        remainder='passthrough'  # passthrough numerical columns as they are
    )
    
    # initialize models
    lgb_model = LGBMRegressor(objective='regression', random_state=42, metric="rmse", learning_rate=0.114, max_depth=41,
                     n_estimators=775, num_leaves=20, reg_lambda=0.123, reg_alpha=0.46)
    xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42, reg_alpha=0.795, reg_lambda=0.172,
                            colsample_bytree=0.836, gamma=0.042, learning_rate=0.114,
                            max_depth=13, subsample=0.920)
    cat_model = CatBoostRegressor(loss_function='RMSE', verbose=False)

    if model_type == 'lgb':
        return make_pipeline(preprocessor, encoder, lgb_model)
    elif model_type == 'xgb':
        return make_pipeline(preprocessor, encoder, xgb_model)
    elif model_type == 'cat':
        return make_pipeline(preprocessor, encoder, cat_model)
    else: 
        return "Model not available"

# Evaluate the model

In [34]:
def evaluate_model(model_type, X_train, y_train):
    if model_type == None:
        model_type = 'lgb'
        
    pipeline = get_estimator(model_type)
    
    # get training time
    start = timer()
    pipeline.fit(X_train, y_train)
    end = timer()
    print('Fitting time on the full training set is %.3f seconds' % (end - start))
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='neg_mean_squared_error')
    print('The cross-validation RMSE is %.5f' % np.mean(np.sqrt(-scores)))

In [35]:
# Catboost Regressor
evaluate_model('cat', X_train, y_train)

Fitting time on the full training set is 6.168 seconds
The cross-validation RMSE is 0.32974


In [37]:
# LGBM Regressor
evaluate_model('lgb', X_train, y_train)

Fitting time on the full training set is 2.914 seconds
The cross-validation RMSE is 0.33545


In [38]:
# XGB Regressor
evaluate_model('xgb', X_train, y_train)

Fitting time on the full training set is 4.095 seconds
The cross-validation RMSE is 0.34550
